In [1]:
import pandas as pd
import plotly.graph_objects as go
import Covid19

area = 'Catalunya'  # Catalunya - Greece - Barcelonès - Attiki - ...
# TODO: add comment with all list of areas

# Create manager object
cov19 = Covid19.Covid19Manager()

# covid19response.gr
# https://github.com/iMEdD-Lab/open-data
# https://github.com/Covid-19-Response-Greece/covid19-data-greece/tree/master/data/greece

# Read Data

In [2]:
# read population
pop = cov19.GetCatalunyaPopulation(mode='comarques')

# read confirmed
confirmed = cov19.GetCatalunyaConfirmed(mode='comarques')

Read data from Catalunya: 144108 registers from 2020-02-25 00:00:00 to 2020-09-06 00:00:00
Filtering by type: 37397 registers
Aggregate by areas: 134836 confirmed cases from 42 areas


In [3]:
# select area
data = confirmed[confirmed.Area == area].reset_index(drop = True)

In [4]:
# compute rolling means
rolling_n = 14
data['Confirmed_rollingmean'] = cov19.ComputeRollingMean(data, 'Confirmed', rolling_n=rolling_n)

In [5]:
# compute epg
data['epg'] = cov19.ComputeEPG(data, 'Confirmed', pop[area])

In [7]:
data.drop(columns=['rho_A', 'rho_B'])

,Date,Area,Confirmed,Confirmed_rollingmean,rho,rho_7,ia_14,epg
0,2020-02-27,Catalunya,1,NaN,NaN,NaN,NaN,NaN
1,2020-02-28,Catalunya,5,NaN,NaN,NaN,NaN,NaN
2,2020-03-02,Catalunya,3,NaN,NaN,NaN,NaN,NaN
3,2020-03-03,Catalunya,7,NaN,NaN,NaN,NaN,NaN
4,2020-03-04,Catalunya,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
186,2020-09-02,Catalunya,1217,1134.785714,0.881621,1.064430,210.003861,223.534336
187,2020-09-03,Catalunya,1299,1127.000000,1.187648,1.050709,208.563034,219.139146
188,2020-09-04,Catalunya,1291,1110.571429,1.579013,1.022710,205.522757,210.190143
189,2020-09-05,Catalunya,467,1106.142857,1.395890,1.004706,204.703203,205.666574


In [184]:
plot_daily(data, column_date='Date', 
           column_value='Confirmed', name_value='Confirmed', color_value='lightskyblue',
           column_rolling='Confirmed_rollingmean', name_rolling='Mean {} days'.format(rolling_n), color_rolling='royalblue'
          )

# plot daily cases
go.Figure(data=[
    go.Bar(
        name=name_value,
        x=data[column_value], y=data.,
        marker_color = 'lightskyblue'
    ),
    go.Scatter(
        name='Mean {} days'.format(rolling_n),
        x=data.Date, y=data.Confirmed_rollingmean,
        marker_color = 'royalblue'        
    )
]).update_layout(
    title="Daily Cases {}".format(area),
    legend=dict(
        orientation='h',
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=0.99),
    yaxis = dict(
        range=(data.Confirmed.min(), data.Confirmed.max()*1.2)
    )
).show()

In [120]:
# plot daily deaths
go.Figure(data=[
    go.Bar(
        name='Deaths',
        x=data.Date, y=data.Deaths,
        marker_color = 'salmon'
    ),
    go.Scatter(
        name='Mean {} days'.format(rolling_n),
        x=data.Date, y=data.Deaths_rollingmean,
        marker_color = 'firebrick'        
    )
]).update_layout(
    title="Daily Deaths",
    legend=dict(
        orientation='h',
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=0.99
    ),
    yaxis = dict(
        range=(0, data.Deaths.max()*1.2)
    )
).show()

In [185]:
# plot risk
go.Figure(data=[
    go.Scatter(
        name='EPG (risk)',
        x=data.Date, y=data.epg,
        marker_color = 'darkslategray'        
    )
]).update_layout(
    title="EPG (COVID Risk): {}".format(area),
    shapes=[
        dict(
            # add green area: low risk epg<30
            type="rect", xref="x", yref="y", x0="2020-01-01", y0=0, x1="2020-12-31", y1=30,
            fillcolor="palegreen", opacity=0.5, layer="below", line_width=0
        ),
        dict(
            # add yellow area: moderate risk 30<epg<70
            type="rect", xref="x", yref="y", x0="2020-01-01", y0=30, x1="2020-12-31", y1=70,
            fillcolor="yellow", opacity=0.4, layer="below", line_width=0
        ), 
        dict(
            # add orange area: high risk 70<epg<100
            type="rect", xref="x", yref="y", x0="2020-01-01", y0=70, x1="2020-12-31", y1=100,
            fillcolor="orange", opacity=0.4, layer="below", line_width=0
        ),
        dict(
            # add red area: very high risk 100<epg
            type="rect", xref="x", yref="y", x0="2020-01-01", y0=100, x1="2020-12-31", y1=1000,
            fillcolor="red", opacity=0.5, layer="below", line_width=0
        )     
    ],
    yaxis = dict(
        range=(data.epg.min(), data.epg.max()*1.2)
    )
).show()

In [186]:
# plot risk
go.Figure(data=[
    go.Scatter(
        name='EPG (risk)',
        x=data.ia_14, y=data.rho_7,
        marker_color = 'darkslategray'        
    )
]).update_layout(
    title="EPG (COVID Risk): {}".format(area),
    width=600, height=600
).show()